# Run scenarios in FaIR

Taking the emissions pathways generated in notebook 100, run the scenarios in FaIR and look at forcing and temperature outputs.

In [ ]:
import json
from multiprocessing import Pool
import platform

from climateforcing.utils import mkdir_p
import h5py
import fair
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# h5 input/output functions : to add to climateforcing.utils

def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    with h5py.File(filename, 'w') as h5file:
        recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
        else:
            raise ValueError('Cannot save %s type'%type(item))

In [ ]:
# this config is appropriate for v1.6.x where x >= 2
fair.__version__

In [ ]:
with open('../data_input_large/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [ ]:
emissions_in = {}
results_out = {}
WORKERS = 3  # set this based on your individual machine - allows parallelisation. nprocessors-1 is a sensible shout.

In [ ]:
scenarios = ["ssp245", "baseline", "hfc-32", "propane", "propane-as-slcf"]

In [ ]:
for scenario in scenarios:
    emissions_in[scenario] = np.loadtxt('../data_output/fair_emissions_files/{}.csv'.format(scenario), delimiter=',')

## convenience function for running FaIR in parallel config with each emission species

In [ ]:
def run_fair(args):
    thisC, thisF, thisT, _, thisOHU, _, thisAF = fair.forward.fair_scm(**args)
    return (thisT, thisF[:,7], thisF[:,9], thisF[:,22], thisF[:,31], np.sum(thisF, axis=1))

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Geoffroy"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False
    #    updated_config[i]["scale"][43] = 0.6
        updated_config[i]["F_solar"][270:] = 0
        
    # multiprocessing is not working for me on Windows
#    if platform.system() == 'Windows':
    shape = (361, len(updated_config))
    t = np.ones(shape) * np.nan
    f_hfc32 = np.ones(shape) * np.nan
    f_hfc125 = np.ones(shape) * np.nan
    f_hcfc22 = np.ones(shape) * np.nan
    f_o3 = np.ones(shape) * np.nan
    f_tot = np.ones(shape) * np.nan
    for i, cfg in tqdm(enumerate(updated_config), total=len(updated_config), position=0, leave=True):
        t[:,i], f_hfc32[:,i], f_hfc125[:,i], f_hcfc22[:,i], f_o3[:,i], f_tot[:,i] = run_fair(updated_config[i])
    
#    else:
#        if __name__ == '__main__':
#            with Pool(WORKERS) as pool:
#                result = list(tqdm(pool.imap(run_fair, updated_config), total=len(updated_config), position=0, leave=True))
#
#        result_t = np.array(result).transpose(1,2,0)
#        t, f_hfc32, f_hfc125, f_hfc22, f_o3, f_tot = result_t
    temp_rebase = t - t[100:151,:].mean(axis=0)
    
    return temp_rebase, f_hfc32, f_hfc125, f_hcfc22, f_o3, f_tot

## Do the runs

In [ ]:
for scenario in tqdm(scenarios, position=0, leave=True):
    results_out[scenario] = {}
    (
        results_out[scenario]['temperature (relative to 1850-1900)'],
        results_out[scenario]['HFC32 radiative forcing'],
        results_out[scenario]['HFC125 radiative forcing'],
        results_out[scenario]['HCFC22 radiative forcing'],
        results_out[scenario]['O3 radiative forcing'],
        results_out[scenario]['total radiative forcing']
    ) = fair_process(emissions_in[scenario])

In [ ]:
mkdir_p('../data_output_large/')
save_dict_to_hdf5(results_out, '../data_output_large/fair-ensemble.h5')